In [2]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
import jupyter_rfb
import mayavi
from mayavi import mlab 
import vispy
# 定义模拟参数
wvl_min = 0.4   # 波长范围
wvl_max = 0.8
fmin = 1/wvl_max
fmax = 1/wvl_min
fcen = 0.5*(fmin+fmax)
df = fmax-fmin

# 定义计算网格
res =10        # 网格分辨率
dpml = 0.5      # PML厚度
sz = 10         # 结构厚度
cell_size = mp.Vector3(1.5, 1.5, dpml+sz+dpml)

# 定义介质和结构
eps_bg = 1    # 背景介质
eps_slab = 1.5 # 结构介质
geometry = [mp.Block(size=mp.Vector3(mp.inf, mp.inf, 6),center=mp.Vector3(0, 0, -2),
                     material=mp.Medium(epsilon=eps_slab))]

# 定义光源和探测器
sources = [mp.Source(mp.GaussianSource(fcen, fwidth=df),
                      component=mp.Ez,
                      center=mp.Vector3(0,0,3),
                      size=mp.Vector3(0.5,0,0))]


# 定义求解器
sim = mp.Simulation(cell_size=cell_size,
                    geometry=geometry,
                    sources=sources,
                    resolution=res,
                    boundary_layers=[mp.PML(dpml)],
                    default_material=mp.Medium(epsilon=eps_bg),
                    symmetries=[mp.Mirror(mp.X)])

detectors = mp.FluxRegion(center=mp.Vector3(0,0,-5),size=mp.Vector3(3,0,3))
trans = sim.add_flux(fcen, df, 100, detectors)
sim.plot3D()
#sim.plot_fields()
mlab.show()
sim.run(until_after_sources=mp.stop_when_fields_decayed(50, mp.Ez, mp.Vector3(0,0,-4.5), 1e-3))

# 计算反射率
flux_freqs = mp.get_flux_freqs(trans)
refl_data = mp.get_fluxes(trans)
#mp.output_epsilon(sim=sim)
# 画图显示结果
plt.figure(figsize=(8,6))
plt.plot(1/np.array(flux_freqs), np.array(refl_data))
plt.xlabel("Frequency (μm^-1)")
plt.ylabel("Reflection")
plt.show()

AttributeError: module 'meep' has no attribute 'Bloch'